In [1]:
!pip install tensorflow_probability

  Using cached tensorflow_probability-0.25.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached dm_tree-0.1.9-cp313-cp313-win_amd64.whl.metadata (2.5 kB)
Using cached tensorflow_probability-0.25.0-py2.py3-none-any.whl (7.0 MB)
Using cached dm_tree-0.1.9-cp313-cp313-win_amd64.whl (102 kB)

   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [tensorflow_probability]
   -------------------- ------------------- 1/2 [te

In [3]:
!pip install tf-keras

  Using cached tf_keras-2.20.1-py3-none-any.whl.metadata (1.8 kB)
Using cached tf_keras-2.20.1-py3-none-any.whl (1.7 MB)


In [7]:
!pip install keras-mdn-layer

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-mdn-layer: filename=keras_mdn_layer-0.3.0-py3-none-any.whl size=7064 sha256=154e6f5670195b101415bda809a79882c2ac27b7488f3945afe3042d993e6b55
  Stored in directory: c:\users\falgun dadhich\appdata\local\pip\cache\wheels\ca\3b\51\b50e5cb62d37846fd7421e54c668f70e3946e31a5a52ea2759
Successfully built keras-mdn-layer


  DEPRECATION: Building 'keras-mdn-layer' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'keras-mdn-layer'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [9]:
# -*- coding: utf-8 -*-
"""
LSTM–MDN for Value-at-Risk forecasting
EXTENDED from indices → NIFTY100 stocks
MODEL TYPES UNCHANGED
"""

# ===============================
# DEPENDENCIES (UNCHANGED)
# ===============================

import numpy as np
import pandas as pd
import random as rn
import os
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import keras
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow_probability import distributions as tfd
import mdn
import warnings
warnings.filterwarnings("ignore")

# ===============================
# REPRODUCIBILITY (UNCHANGED)
# ===============================

os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(6969)
rn.seed(6969)
tf.random.set_seed(6969)

# ===============================
# PARAMETERS (UNCHANGED)
# ===============================

d = 10
test_start_date = "2021-01-01"
test_end_date   = "2022-12-31"

save_results = True
period_appendix = "_nifty100"

tensorboard = TensorBoard(log_dir='tbdir/', histogram_freq=0)
mon = EarlyStopping(monitor='loss', patience=5)

# ===============================
# LOAD NIFTY100 STOCK LIST
# ===============================

stocks = pd.read_csv("data/nifty100_constituents.csv")["Symbol"].tolist()

# ===============================
# UTILITIES (UNCHANGED)
# ===============================

def df_to_X_y(df, window_size=10):
    df_as_np = df.to_numpy()
    X, y = [], []
    for i in range(len(df_as_np) - window_size):
        X.append([[a] for a in df_as_np[i:i+window_size]])
        y.append(df_as_np[i + window_size])
    return np.array(X), np.array(y)

# ===============================
# MDN LOSS (UNCHANGED)
# ===============================

def get_mixture_loss_func_REGULARIZED(output_dim, num_mixes, lambda_reg=0.2):
    def mdn_loss_func(y_true, y_pred):
        y_pred = tf.reshape(y_pred, [-1, (2*num_mixes*output_dim)+num_mixes])
        y_true = tf.reshape(y_true, [-1, output_dim])

        out_mu, out_sigma, out_pi = tf.split(
            y_pred,
            [num_mixes*output_dim, num_mixes*output_dim, num_mixes],
            axis=-1
        )

        cat = tfd.Categorical(logits=out_pi)
        mus = tf.split(out_mu, num_mixes, axis=1)
        sigs = tf.split(out_sigma, num_mixes, axis=1)
        components = [
            tfd.MultivariateNormalDiag(loc=m, scale_diag=s)
            for m, s in zip(mus, sigs)
        ]

        mixture = tfd.Mixture(cat=cat, components=components)
        loss = -tf.reduce_mean(mixture.log_prob(y_true))
        loss += lambda_reg * tf.reduce_sum(tf.square(out_pi))
        return loss
    return mdn_loss_func

# ===============================
# MDN OUTPUT PARSING (UNCHANGED)
# ===============================

def get_outputs(predictions, N_MIXES=2):
    mus = predictions[:, :N_MIXES]
    sigs = predictions[:, N_MIXES:2*N_MIXES]
    pis = mdn.softmax(predictions[:, -N_MIXES:])
    return mus, sigs, pis

def MDN_predict(model, test_data, N_MIXES=2):
    preds = model.predict(test_data)
    mu, sigma, pi = get_outputs(preds, N_MIXES)
    return {"mu": mu, "sigma": sigma, "pi": pi}

def dataframe_converter(pred, N_MIXES=2):
    cols = []
    for i in range(1, N_MIXES+1): cols.append(f"mu{i}")
    for i in range(1, N_MIXES+1): cols.append(f"sigma{i}")
    for i in range(1, N_MIXES+1): cols.append(f"pi{i}")
    return pd.DataFrame(
        np.column_stack([pred["mu"], pred["sigma"], pred["pi"]]),
        columns=cols
    )

# ===============================
# MAIN LOOP — PER STOCK
# ===============================

for stock in stocks:

    print(f"Training MDN models for {stock}")

    df = pd.read_csv(f"data/prices/{stock}.csv")
    df["date"] = pd.to_datetime(df["date"])

    test_start_idx = df[df["date"] > test_start_date].index.min() - d
    test_data = df.iloc[test_start_idx:]
    train_data = df.iloc[:test_start_idx]

    X_train, y_train = df_to_X_y(train_data["R"], d)
    X_test, y_test   = df_to_X_y(test_data["R"], d)

    # ===============================
    # MODEL 1 — 2 MIX (VANILLA)
    # ===============================

    model_plain = Sequential([
        InputLayer(input_shape=(d,1)),
        LSTM(6, activation="relu"),
        Dense(10, activation="relu"),
        mdn.MDN(1, 2)
    ])

    model_plain.compile(
        loss=mdn.get_mixture_loss_func(1,2),
        optimizer=keras.optimizers.Adam()
    )

    model_plain.fit(
        X_train, y_train,
        epochs=100,
        callbacks=[mon, tensorboard],
        verbose=0
    )

    # ===============================
    # MODEL 2 — 2 MIX REGULARIZED
    # ===============================

    model_reg = Sequential([
        InputLayer(input_shape=(d,1)),
        LSTM(6, activation="relu"),
        Dense(10, activation="relu"),
        mdn.MDN(1, 2)
    ])

    model_reg.compile(
        loss=get_mixture_loss_func_REGULARIZED(1,2,0.1),
        optimizer=keras.optimizers.Adam()
    )

    model_reg.fit(
        X_train, y_train,
        epochs=100,
        callbacks=[mon, tensorboard],
        verbose=0
    )

    # ===============================
    # MODEL 3 — 3 MIX (FINAL MODEL)
    # ===============================

    model_C3 = Sequential([
        InputLayer(input_shape=(d,1)),
        LSTM(6, activation="relu"),
        Dense(10, activation="relu"),
        mdn.MDN(1, 3)
    ])

    model_C3.compile(
        loss=mdn.get_mixture_loss_func(1,3),
        optimizer=keras.optimizers.Adam()
    )

    model_C3.fit(
        X_train, y_train,
        epochs=100,
        callbacks=[mon, tensorboard],
        verbose=0
    )

    # ===============================
    # SAVE RESULTS (UNCHANGED FORMAT)
    # ===============================

    if save_results:
        out = dataframe_converter(
            MDN_predict(model_C3, X_test, 3),
            3
        )
        out.to_csv(
            f"results/mdn_outputs/{stock}_C3{period_appendix}.csv",
            index=False
        )

    tf.keras.backend.clear_session()


ModuleNotFoundError: No module named 'tensorflow.compat.v1.keras'